In [11]:
import pickle
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy import special                 
import array
import scipy as sp
from scipy import spatial
from scipy.stats import binned_statistic
import re

In [12]:
def distance(points1, points2):
    return(np.sqrt((points1[0]-points2[0])**2 +(points1[1]-points2[1])**2+(points1[2]-points2[2])**2))
def nfvir(lyman_distance,n,v1,v2,v3):
    gals = np.loadtxt('hprops.res', usecols=[5,6,7,8],unpack=1) 
    points = []
    for i,j,k in zip(gals[0],gals[1],gals[2]):
        points.append([i,j,k])
    radius_points = []
    for i in gals[3]:
        radius_points.append(i)
    #lls in rvir
    #total lls
    #bins count
    #one extra space
    points.append([0,0,0])
    lymans = []
    neutral_fraction =[]
    HI= []
    gas_density_normalized =[]
    count = 0
    for i in (lyman_distance):
        #i[0] is the column density, i[1] is the lyman location
        #less than certain density in bins
        
        for z in range(n,0,-1):
            points[-1] = (i[1])
            tree = spatial.KDTree(points)
            #look at what is associated
            all_nn_indices = tree.query_ball_point(points[-1],r=4.979e-01*z)
            #check fvir conditions
            condition = False
            for k in all_nn_indices:
                if(points[k] != points[-1]):
                #see if distance in between is smaller than vir distance times by n 
                #add mass constraints
                    if(distance(i[1],points[k]) < radius_points[k]*z):
                        condition = True
                        #if true
            if condition == True:
                #range,lyman
                lymans.append([i,z])
                neutral_fraction.append([v1[count]])
                HI.append([v2[count]])
                gas_density_normalized.append([v3[count]])
        count += 1
    return lymans,neutral_fraction,HI,gas_density_normalized

In [13]:
#need the column density, and lyman distance

In [14]:
def unPackRawFile(raw_path):
    """


    
    - unpacks the .raw file. Not used for the neural network.
    """
    y = np.loadtxt(raw_path, skiprows=6)
    distance = y[:,0]
    pec_vel = y[:,1]
    temp = y[:,2]
    HI_density = y[:,3]
    gas_density = y[:,4]
    gas_metallicity = y[:,5]
   
    return distance, pec_vel, temp, HI_density, gas_density, gas_metallicity

In [15]:
def calculate_gamma(temperature,density,xHI,scale):
    '''Gamma = R(T)*ne/xHI, R(T) = 4.3e-13 * (T/1e4K)^{-0.7), ne = 0.82*nb.
    '''
    omega_b = 0.0479; h = 0.6814
    mean_nb = 1.123e-5/scale**3*omega_b*h**2
    nb = np.multiply(density ,mean_nb); ne = 0.82*nb
    R_T = 4.3e-13 * (np.divide(temperature,1e4))**(-0.7)
    gamma = R_T * ne / xHI
    
    return gamma

In [16]:
def getPos(path_LOS,linenumber=7):
    """
    the start position of the LOS is given inside each file, (in the comments)
    this function parses the comments to get that information
    """
    f = open(path_LOS)
    x = f.readlines()[linenumber]
    answer = re.search('\(([^)]+)', x).group(1)
    arr = np.array(answer.split(','),dtype=float)
    return arr

def getDir(path_LOS,linenumber=7):
    """
    the direction of the LOS is given inside each file, (in the comments)
    this function parses the comments to get that information
    """
    f = open(path_LOS)
    x = f.readlines()[linenumber]
    answer = re.search('\(([^)]+)', x.split(', ')[1]).group(1)
    arr = np.array(answer.split(','),dtype=float)
    return arr


def convertSphereToCart(theta, phi):
    "converts a unit vector in spherical to cartesian, needed for getGalaxies"
    return np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])

In [17]:
def lyman_identification():
    """
    function for identifying lls using neutral fraction as filter
    """
    neutral_fraction_limit = 1e-3
    
    neutral_fraction = []
    real_lyman = []
    gamma = []
    HI = []
    gas_den = []
    for i in range(1000):
        raw_path = 'los.00' + '{0:03}'.format(i) +'.raw'
        distance, pec_vel, temp, HI_density, gas_density, gas_metallicity = unPackRawFile(raw_path)
        #changing the distance from chimp to cmeters
        distance_cm = distance *0.1452*3.086e24 /0.68
        position = getPos(raw_path)
        direction = convertSphereToCart(getDir(raw_path)[0],getDir(raw_path)[1])
        for k in range(len(distance) -2):
            if(HI_density[k]/gas_density[k]<neutral_fraction_limit and HI_density[k+1]/gas_density[k+1]>neutral_fraction_limit):
                left_bases = k+1
                peak_fraction = HI_density[k]/gas_density[k]
                peak_HI = HI_density[k]
                peak_distance = distance[k]
                peak_gas = gas_density[k]
                if( k+1 <len(distance)):
                    while(not(HI_density[k]/gas_density[k]>neutral_fraction_limit and HI_density[k+1]/gas_density[k+1]<neutral_fraction_limit) and k <len(distance)-1):
                        right_bases = k +1
                        if(k<len(distance)-1):
                            k+=1
                        if(HI_density[k]/gas_density[k]>peak_fraction):
                            peak_fraction = HI_density[k]/gas_density[k]
                            peak_HI = HI_density[k]
                            peak_distance = distance[k]
                            peak_gas = gas_density[k]
                            peak_gamma = calculate_gamma(temp[k],gas_density[k],peak_fraction,0.1452)
                    area_under = peak_HI *1/2*(distance_cm[right_bases]-distance_cm[left_bases])/0.937415
                    real_distance = [(position[0]/512*80+peak_distance*direction[0])%80, (position[1]/512*80+peak_distance*direction[1])%80,(position[2]/512*80+peak_distance*direction[2])%80]
                    if(area_under >= (10**16)and area_under  <= (10**22)):
                        real_lyman.append([area_under,real_distance])
                        neutral_fraction.append(peak_fraction)
                        HI.append(peak_HI)
                        gamma.append(peak_gamma)
                        gas_den.append(peak_gas)
    return real_lyman,neutral_fraction,gamma,HI,gas_den
lyman_limited_value,neutral_fraction,gamma,HI,gas_density_normalized = lyman_identification()


In [26]:
lymans,n_fraction,H1,gas_den_normalized = nfvir(lyman_limited_value,4,neutral_fraction,HI,gas_density_normalized)

In [72]:
with open('a.pkl', 'wb') as f:
    pickle.dump(lymans, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('b.pkl', 'wb') as f:
    pickle.dump(n_fraction, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('c.pkl', 'wb') as f:
    pickle.dump(H1, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('d.pkl', 'wb') as f:
    pickle.dump(gas_den_normalized, f, protocol=pickle.HIGHEST_PROTOCOL)

In [75]:
lymans= []
n_fraction = []
with open('a.pkl', 'rb') as f:
    lymans = pickle.load(f)
with open('b.pkl', 'rb') as f:
    n_fraction = pickle.load(f)

In [76]:
no_repeat =[]
neutral_fraction_stuff = []
for i in range(0,len(lymans)-1):
    if(lymans[i][0][0] != lymans[i+1][0][0]):
        no_repeat.append(lymans[i])
        neutral_fraction_stuff.append(n_fraction[i])

In [77]:
lyman_temp  = []
for i in no_repeat:
    lyman_temp.append(i[0][0])

In [78]:
color_value = []
for i in no_repeat:
    color_value.append(i[1])

In [79]:
sigma = 6.3e-18

In [80]:
optical_depth = np.multiply(sigma,lyman_temp)